INTEGRANTES: BRUNO FIGUEROA

En esta tarea, deberá comparar distintos algoritmos de aprendizaje supervisado y no supervisado sobre un mismo conjunto de datos de su elección.

Su dataset deberá tener al menos 10.000 filas y 7 columnas utilizables.

Además, seleccione una de las columnas como su etiqueta Y, la cual debe ser discreta y con más de 2 clases.

PUNTO 1:

Deberá ejecutar tres algoritmos de clustering: K-Means, K-Means++ y MeanShift, utilizando al menos cuatro configuraciones distintas para cada técnica (deberá justificar la elección de parámetros).

Para el entrenamiento, use únicamente el 80% de los datos, omitiendo la etiqueta Y. 

A continuación, evalúe las doce configuraciones obtenidas mediante una métrica de su elección (por ejemplo, Silhouette Score) y seleccione las tres de mejor desempeño. 

Luego, aplique cada una de estas configuraciones al 20% restante de los datos, asignando a cada muestra el cluster correspondiente (obtenidos desde el entrenamiento). 

Finalmente, compare la etiqueta Y real de cada muestra con la etiqueta dominante dentro del cluster al que pertenece y analice si este procedimiento resulta razonable para asignar etiquetas faltantes.

Instalar dependencias

In [4]:
!pip install pandas scikit-learn

Importar librerias para punto 1 y 2

In [5]:
from sklearn.datasets import fetch_covtype
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle as skshuffle

from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np
import json
import time

Cargar dataset como DataFrame y sacar muestra aleatoria de 10k filas. 

Luego verificar estructura.

Y finalmente crear 2 vectores, uno para parametros (X), y otro para etiquetas (Y).

In [6]:
data = fetch_covtype(as_frame=True)
df = data.frame
df = df.sample(n=10000, random_state=42)

print(df.shape)
print(df['Cover_Type'].value_counts())

X = df.drop(columns=['Cover_Type'])
y = df['Cover_Type']

results = []

(10000, 55)
Cover_Type
2    4841
1    3683
3     623
7     347
6     299
5     160
4      47
Name: count, dtype: int64


K-Means.

Usando una proporcion 80-20 con 4 configuraciones diferentes.

Cada metodo tiene una evaluacion como silhouette score, esto para comparalos al final.

In [7]:
print()
print("K-MEANS")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = tts(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

kmeans_configs = [
    {"n_clusters": 3, "init": "random", "n_init": 10, "max_iter": 300},
    {"n_clusters": 5, "init": "random", "n_init": 15, "max_iter": 300},
    {"n_clusters": 7, "init": "random", "n_init": 20, "max_iter": 500},
    {"n_clusters": 9, "init": "random", "n_init": 25, "max_iter": 400},
]

for i, cfg in enumerate(kmeans_configs, 1):
    model = KMeans(**cfg, random_state=42)
    model.fit(X_train)
    labels = model.labels_

    sil_score = silhouette_score(X_train, labels)
    results.append({
        "model_type": "KMeans",
        "config": cfg,
        "silhouette": sil_score
    })
    print(f"Config {i}: {cfg}, Silhouette Score = {sil_score:.4f}")


K-MEANS
Config 1: {'n_clusters': 3, 'init': 'random', 'n_init': 10, 'max_iter': 300}, Silhouette Score = 0.1021
Config 2: {'n_clusters': 5, 'init': 'random', 'n_init': 15, 'max_iter': 300}, Silhouette Score = 0.0807
Config 3: {'n_clusters': 7, 'init': 'random', 'n_init': 20, 'max_iter': 500}, Silhouette Score = 0.1387
Config 4: {'n_clusters': 9, 'init': 'random', 'n_init': 25, 'max_iter': 400}, Silhouette Score = 0.1700


K-Means++. (init:k-means++)

Usando una proporcion 80-20 con 4 configuraciones diferentes.

Cada metodo tiene una evaluacion como silhouette score, esto para comparalos al final.

In [8]:
print()
print("K-MEANS++")

kmeanspp_configs = [
    {"n_clusters": 3, "init": "k-means++", "n_init": 10, "max_iter": 300},
    {"n_clusters": 5, "init": "k-means++", "n_init": 20, "max_iter": 300},
    {"n_clusters": 7, "init": "k-means++", "n_init": 15, "max_iter": 500},
    {"n_clusters": 9, "init": "k-means++", "n_init": 25, "max_iter": 400},
]

for i, cfg in enumerate(kmeanspp_configs, 1):
    model = KMeans(**cfg, random_state=42)
    model.fit(X_train)
    labels = model.labels_
    sil_score = silhouette_score(X_train, labels)
    results.append({
        "model_type": "KMeans++",
        "config": cfg,
        "silhouette": sil_score
    })
    print(f"Config {i}: {cfg}, Silhouette Score = {sil_score:.4f}")


K-MEANS++
Config 1: {'n_clusters': 3, 'init': 'k-means++', 'n_init': 10, 'max_iter': 300}, Silhouette Score = 0.1021
Config 2: {'n_clusters': 5, 'init': 'k-means++', 'n_init': 20, 'max_iter': 300}, Silhouette Score = 0.1154
Config 3: {'n_clusters': 7, 'init': 'k-means++', 'n_init': 15, 'max_iter': 500}, Silhouette Score = 0.0970
Config 4: {'n_clusters': 9, 'init': 'k-means++', 'n_init': 25, 'max_iter': 400}, Silhouette Score = 0.1159


MeanShift

Usando una proporcion 80-20 con 4 configuraciones diferentes.

Cada metodo tiene una evaluacion como silhouette score, esto para comparalos al final.

In [9]:
print()
print("MeanShift")

bandwidth_estimate = estimate_bandwidth(X_train, quantile=0.2, n_samples=500)
print(f"Bandwidth estimado base: {bandwidth_estimate:.4f}")

meanshift_configs = [
    {"bandwidth": bandwidth_estimate * 0.5},
    {"bandwidth": bandwidth_estimate},
    {"bandwidth": bandwidth_estimate * 1.5},
    {"bandwidth": bandwidth_estimate * 2.0},
]

for i, cfg in enumerate(meanshift_configs, 1):
    model = MeanShift(**cfg)
    model.fit(X_train)
    labels = model.labels_

    if len(np.unique(labels)) > 1:
        sil_score = silhouette_score(X_train, labels)
    else:
        sil_score = -1

    results.append({
        "model_type": "MeanShift",
        "config": cfg,
        "silhouette": sil_score
    })
    print(f"Config {i}: {cfg}, Silhouette Score = {sil_score:.4f}, Clusters encontrados = {len(np.unique(labels))}")


MeanShift
Bandwidth estimado base: 7.5653
Config 1: {'bandwidth': np.float64(3.782648713514269)}, Silhouette Score = 0.3996, Clusters encontrados = 84
Config 2: {'bandwidth': np.float64(7.565297427028538)}, Silhouette Score = 0.3058, Clusters encontrados = 26
Config 3: {'bandwidth': np.float64(11.347946140542806)}, Silhouette Score = 0.4296, Clusters encontrados = 21
Config 4: {'bandwidth': np.float64(15.130594854057076)}, Silhouette Score = 0.5170, Clusters encontrados = 16


JUSTIFICACION DE PARAMETROS:

En K-Means y K-Means++, se ajustaron los parametros "n_clusters", "n_init" y "max_iter".

Donde n_clusters es el numero de clusters con los que se realiza el modelo, se utilizaron valores como 3 5 7 9, intentando probar con cantidades no muy grandes, ya que si se utilizan 20, 100 o 1000, se puede llegar a un punto de demasiada segmentacion, los cuales comparten la misma informacion, y solo no se unieron por distancia.

n_init es el numero de inicializaciones independientes para reducir el riesgo de estancarse en minimos locales, estos valores van de 10 a 25, y fue incrementado junto con el numero de clusters.

Finalmente max_iter es el numero de iteraciones por cada iteracion, aumentar este valor permite una convergencia mas precisa, aunque en un modelo bien configurado y con un dataset bien balanceado, esta convergencia nunca deberia tomar mas de 300 iteraciones. 

En MeanShift, se calcula el modelo utilizando el parametro "bandwith" que es la distancia en la que se conectan los diferentes puntos, por ende, se crea un aproximado base usando la formula "estimate_bandwidth", y se intenta con multiplos de ese valor (0.5; 1; 1.5; 2), no se intenta con valores muy grandes o pequeños debido a que el sistema convergeria a todos los puntos en un unico grupo, o a que se creen montones de pequeños grupos con la misma informacion.

Evaluación global de las 12 configuraciones:

Se toman los 12 modelos entrenados anteriormente, y se ordenan en base a su puntaje de silhuette, obteniendo el top 3.

Luego, se compara la informacion de etiqueta, con la etiqueta real usando el resto (20%) del dataset base, entregando una tasa de aciertos final por cada modelo en el top 3.

In [10]:
results = sorted(results, key=lambda x: x["silhouette"], reverse=True)
top3 = results[:3]

print()
print("Top 3 configuraciones globales por Silhouette Score")
for res in top3:
    print(res)

for i, res in enumerate(top3, 1):
    algo = res["model_type"]
    cfg = res["config"]
    print(f"\nModelo {i}: {algo}, Config: {cfg}")

    if algo in ["KMeans", "KMeans++"]:
        model = KMeans(**cfg, random_state=42)
    else:
        model = MeanShift(**cfg)

    model.fit(X_train)
    test_clusters = model.predict(X_test)

    train_clusters = model.labels_
    cluster_labels = {}
    for cluster_id in np.unique(train_clusters):
        mask = train_clusters == cluster_id
        dominant_label = y_train.iloc[mask].mode()[0]
        cluster_labels[cluster_id] = dominant_label

    y_pred = [cluster_labels[c] for c in test_clusters if c in cluster_labels]
    valid_idx = [i for i, c in enumerate(test_clusters) if c in cluster_labels]
    match_ratio = np.mean(np.array(y_pred) == y_test.values[valid_idx])
    print()
    print(f"Coincidencia entre etiquetas reales y dominantes = {match_ratio:.4f}")


Top 3 configuraciones globales por Silhouette Score
{'model_type': 'MeanShift', 'config': {'bandwidth': np.float64(15.130594854057076)}, 'silhouette': 0.5170171193037753}
{'model_type': 'MeanShift', 'config': {'bandwidth': np.float64(11.347946140542806)}, 'silhouette': 0.4296315428885252}
{'model_type': 'MeanShift', 'config': {'bandwidth': np.float64(3.782648713514269)}, 'silhouette': 0.3996245643432745}

Modelo 1: MeanShift, Config: {'bandwidth': np.float64(15.130594854057076)}

Coincidencia entre etiquetas reales y dominantes = 0.4920

Modelo 2: MeanShift, Config: {'bandwidth': np.float64(11.347946140542806)}

Coincidencia entre etiquetas reales y dominantes = 0.4960

Modelo 3: MeanShift, Config: {'bandwidth': np.float64(3.782648713514269)}

Coincidencia entre etiquetas reales y dominantes = 0.6350


Finalmente, se observan los siguientes resultados:

TOP1: Meanshift con 2*bandwith, obtuvo un 51.7% de silhouette score y una coincidencia de 49.20% con las etiquetas reales.

TOP2: Meanshift con 1.5*bandwith, alcanzó un silhouette score de 0.43 y una coincidencia de 49.60%, resultado equivalente al del top 1.

TOP3: Meanshift con 0.5*bandwitch, logró un silhouette score de 0.39 y una coincidencia de 63.50%, siendo el mejor entre los tres.

Estos datos, demuestran como el modelo esta funcionando correctamente, siendo capaz de predecir hasta cierto punto el resultado, llegando hasta casi un 50/50 en el TOP1 y TOP2, y con un 64% en el mejor caso.

Teniendo en cuenta que la etiqueta tiene 7 clases, estos modelos permiten pasar de un 14% de exito (seleccion aleatoria) a un 49% o 64%, segun el modelo utilizado.

Ahora, respondiendo la pregunta ¿Este procedimiento resulta razonable para asignar etiquetas faltantes?

La respuesta es un depende, ya que en el mejor de los casos, se obtuvo solo un rendimiento del 64%, lo que no se puede considerar demasiado confliable, por ende, se deberia buscar un metodo algo mas exacto (rondando el 85+% de aciertos) para que sea razonable de aplicar, en caso de no tener disponible otro metodo, si es razonable el utilizarlo, ya que al ser un 64%, el modelo permite predecir la mayoria de situaciones, en vez de un modelo puramente aleatorio con un 14%.

PUNTO 2:

Utilizando el mismo conjunto de datos previamente seleccionado, diseñe e implemente una técnica que permita entrenar en paralelo múltiples instancias (al menos tres por técnica) de Regresión Logística y SVM, variando sus hiperparámetros (por ejemplo: batch size, tasa de aprendizaje, etc, según sea el caso). 

Los parámetros de cada configuración deberán definirse en un archivo de configuración externo, y el entrenamiento deberá realizarse utilizando el 80% de los datos. 

Cada modelo se evaluará periódicamente (solo con datos de entrenamiento), y por cada cinco épocas deberá descartarse la configuración con peor desempeño entre todas las configuraciones restantes. Para las dos mejores configuraciones, presente métricas de evaluación utilizando el conjunto de testing, es decir, el 20% de los datos. 

Analice los resultados obtenidos en función de los hiperparámetros seleccionados.

Cargar configuracion desde archivo externo (configs.json) tiene que estar en el mismo directorio que el .ipynb

In [11]:
with open("configs.json", "r") as f:
    configs = json.load(f)["models"]

print(f"Configuraciones cargadas: {len(configs)} modelos\n")

Configuraciones cargadas: 6 modelos



Subdividir parte del train para evaluacion periodica, y creacion de los modelos.

In [12]:
X_train_main, X_train_eval, y_train_main, y_train_eval = tts(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)
classes = np.unique(y_train)


models_meta = []
for cfg in configs:
    loss = "log_loss" if cfg["type"] == "logistic" else "hinge"
    clf = SGDClassifier(
        loss=loss,
        penalty="l2",
        alpha=cfg["alpha"],
        learning_rate=cfg["learning_rate"],
        eta0=cfg["eta0"],
        random_state=42
    )
    models_meta.append({
        "name": cfg["name"],
        "type": cfg["type"],
        "cfg": cfg,
        "clf": clf,
        "epochs_done": 0,
        "max_epochs": cfg["max_epochs"],
        "batch_size": cfg["batch_size"],
        "alive": True,
        "last_eval_acc": None
    })

Inicializar pesos, y definir la funcion de entrenamiento por bloque.

In [13]:
for m in models_meta:
    init_batch = min(100, X_train_main.shape[0])
    m["clf"].partial_fit(X_train_main[:init_batch], y_train_main[:init_batch], classes=classes)


def train_chunk(model, X_main, y_main, epochs_chunk, batch_size, classes):
    n = X_main.shape[0]
    for ep in range(epochs_chunk):
        X_sh, y_sh = skshuffle(X_main, y_main, random_state=int(time.time() * 1000) % 2**32)
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            Xb, yb = X_sh[start:end], y_sh[start:end]
            model.partial_fit(Xb, yb, classes=classes)
    y_pred_eval = model.predict(X_train_eval)
    acc = accuracy_score(y_train_eval, y_pred_eval)
    return model, acc

Entrenamiento iterativo en paralelo con eliminacion del peor cada 5 epocas.

In [14]:
epochs_chunk = 5
max_rounds = max(m["max_epochs"] for m in models_meta) // epochs_chunk + 1
print(f"Iniciando entrenamiento paralelo ({len(models_meta)} configuraciones, {epochs_chunk} épocas por ronda)\n")

for round_i in range(max_rounds):
    alive = [m for m in models_meta if m["alive"] and m["epochs_done"] < m["max_epochs"]]
    if len(alive) <= 2:
        print("Menos de 3 modelos activos, deteniendo eliminaciones.")
        break

    print(f"--- Ronda {round_i+1} | modelos activos: {len(alive)} ---")
    futures = {}
    with ThreadPoolExecutor(max_workers=min(len(alive), 4)) as exe:
        for m in alive:
            futures[exe.submit(train_chunk, m["clf"], X_train_main, y_train_main, epochs_chunk, m["batch_size"], classes)] = m
        for fut in as_completed(futures):
            m = futures[fut]
            clf_updated, acc = fut.result()
            m["clf"] = clf_updated
            m["epochs_done"] += epochs_chunk
            m["last_eval_acc"] = acc
            print(f"{m['name']} ({m['type']}) -> acc_train_eval={acc:.4f}")

    # Eliminar el peor
    alive = [m for m in models_meta if m["alive"]]
    if len(alive) <= 2:
        break
    worst = min(alive, key=lambda x: x["last_eval_acc"] or -1.0)
    worst["alive"] = False
    print(f"Eliminado: {worst['name']} ({worst['last_eval_acc']:.4f})\n")

# Seleccionar ganadores
finalists = sorted([m for m in models_meta if m["alive"]], key=lambda x: x["last_eval_acc"] or 0.0, reverse=True)[:2]
print("\nFinalistas:")
for f in finalists:
    print(f"- {f['name']} ({f['type']}) acc_train_eval={f['last_eval_acc']:.4f}")

Iniciando entrenamiento paralelo (6 configuraciones, 5 épocas por ronda)

--- Ronda 1 | modelos activos: 6 ---
log_B (logistic) -> acc_train_eval=0.7050
svm_A (svm) -> acc_train_eval=0.6987
log_A (logistic) -> acc_train_eval=0.7075
svm_B (svm) -> acc_train_eval=0.6963
log_C (logistic) -> acc_train_eval=0.6900
svm_C (svm) -> acc_train_eval=0.6837
Eliminado: svm_C (0.6837)

--- Ronda 2 | modelos activos: 5 ---
log_B (logistic) -> acc_train_eval=0.7000
svm_A (svm) -> acc_train_eval=0.6975
log_A (logistic) -> acc_train_eval=0.7087
svm_B (svm) -> acc_train_eval=0.7163
log_C (logistic) -> acc_train_eval=0.6913
Eliminado: log_C (0.6913)

--- Ronda 3 | modelos activos: 4 ---
log_B (logistic) -> acc_train_eval=0.7063
svm_B (svm) -> acc_train_eval=0.7087
log_A (logistic) -> acc_train_eval=0.7087
svm_A (svm) -> acc_train_eval=0.7137
Eliminado: log_B (0.7063)

--- Ronda 4 | modelos activos: 3 ---
log_A (logistic) -> acc_train_eval=0.6975
svm_A (svm) -> acc_train_eval=0.7087
svm_B (svm) -> acc_trai

Evaluar finalistas, utilizando los datos restantes (20%)

In [15]:
results_summary = []
for f in finalists:
    clf = f["clf"]
    y_pred = clf.predict(X_test)
    acc_test = accuracy_score(y_test, y_pred)
    print(f"\n== {f['name']} ({f['type']}) ==")
    print(f"Accuracy test: {acc_test:.4f}")
    print(classification_report(y_test, y_pred, digits=4, zero_division=0))
    results_summary.append({
        "name": f["name"],
        "type": f["type"],
        "train_eval_acc": f["last_eval_acc"],
        "test_acc": acc_test
    })

results_summary


== svm_A (svm) ==
Accuracy test: 0.7040
              precision    recall  f1-score   support

           1     0.7246    0.6282    0.6730       737
           2     0.7126    0.8326    0.7680       968
           3     0.6203    0.7840    0.6926       125
           4     0.0000    0.0000    0.0000         9
           5     0.0000    0.0000    0.0000        32
           6     0.2222    0.0667    0.1026        60
           7     0.7400    0.5362    0.6218        69

    accuracy                         0.7040      2000
   macro avg     0.4314    0.4068    0.4083      2000
weighted avg     0.6829    0.7040    0.6875      2000


== svm_B (svm) ==
Accuracy test: 0.7145
              precision    recall  f1-score   support

           1     0.7064    0.6988    0.7026       737
           2     0.7400    0.8058    0.7715       968
           3     0.6102    0.8640    0.7152       125
           4     0.0000    0.0000    0.0000         9
           5     0.0000    0.0000    0.0000       

[{'name': 'svm_A',
  'type': 'svm',
  'train_eval_acc': 0.70875,
  'test_acc': 0.704},
 {'name': 'svm_B',
  'type': 'svm',
  'train_eval_acc': 0.7025,
  'test_acc': 0.7145}]

Analisis de resultados:

Los 2 mejores modelos de regresion fueron: smv_A y svm_B, ambos llegaron a un desempeño final muy similar de 70.4% y 71.4% de aciertos sobre el conjunto de prueba.

El modelo SVM_A logro un rendimiento ligeramente superior, especialmente la clase 2, que es a mas representada en el conjunto de datos. Ambos modelos lograron identificar correctamente las clases mas frecuentes (1, 2, 3), pero fallaron con las clases menos presentes (4, 5, 6), donde la cantidad de ejemplos disponibles fue demasiado baja para que se pudiera identificar algun patron predictivo, las 3 menores clases juntas, conforman alrededor de un 5% de los datos totales, lo que muestra un claro desbalance en el dataset utilizado.

El modelo SVM_B, aunque con una accuracy ligeramente menor, mostro un puntaje f1 superior, lo que sugiere una convergencia mas estable durante el entrenamiento.

En resumen, los resultados fueron coherentes con respecto a los datos utilizados y los parametros seleccionados, donde este fue incapaz de converger a una solucion para todas las clases de la etiqueta, debido a el desbalance del dataset utilizado, lo que muestra la importancia de aplicar tecnicas de balanceo de datos o ponderacion por clase en casos futuros.

Finalmente, es necesario el mencionar que para esta tarea se utilizo Inteligencia Artificial, especificamente ChatGPT para la creacion del codigo inicial, luego (debido a la incompetencia de este para todo proyecto mayor a 100~ lineas de codigo) fue necesario el debugear y resolver el resto de problemas relacionados al codigo manualmente.